# Harry's House - analyze emotion

In questo notebook andiamo ad analizzare le emozioni che sono state ricavate precedentemente dal notebook **Harry's House - Streaming Api**. Nello specifico andiamo a creare un csv che riporti la somma delle emozioni provate, per ogni minuto dalle 6.03 fino alle 8.59.

In [64]:
!pip install pytz

In [1]:
import tweepy
import CONFIG
import csv
import os
import json
import numpy as np
import pandas as pd
import time
import re
import pytz

from collections import Counter
from datetime import datetime
from zoneinfo import ZoneInfo

In [2]:
path = os.getcwd() + '\HARRY_HOUSE_DATA'
os.chdir(path)
print(os.getcwd())

C:\Users\sarac\OneDrive - Università degli Studi di Milano\UNI 2022\_SOCIALMEDIA\#harryhouse\HARRY_HOUSE_DATA


In [3]:
#lettura del file contentente tutte le emozioni

list_emotions = []

with open('emotion_streaming_api.csv', 'r', encoding='utf-8') as file:
    i = 0
    reader = csv.reader(file)
    for row in reader:
        if i == 0:
            i += 1
            continue
        list_emotions.append(row)

In [4]:
#conversione del tempo in timezone europea

date_format = "%Y-%m-%dT%H:%M:%S.%fZ" 
timezone = pytz.timezone("Europe/Berlin")
target_tz = pytz.timezone('UTC')

In [5]:
#creo dizionario con orari e modifico la lista emozioni con nuovo orario

dict_emotions = {}
list_emotions_modified = []

for s in list_emotions:
    #sistemazione formato data
    string = s[0]
    date = datetime.strptime(string, date_format)
    date = target_tz.localize(date)
    date = timezone.normalize(date)
    time = date.time()
    time = time.replace(second = 0)
    
    new_list = [time, s[1], s[2]]
    list_emotions_modified.append(new_list)
    
    e = {
        'joy': 0,
        'anger': 0,
        'surprise': 0,
        'sadness': 0,
        'fear': 0
    }
    dict_emotions[time] = e

In [6]:
#conteggio delle varie emozioni 

def count_emotions(dict_emotions, emotion, time):
    sub_dict = dict_emotions.get(time)
   
    if emotion == 'joy':
        sub_dict['joy'] = sub_dict.get('joy') + 1
        dict_emotions[time] = sub_dict 
    elif emotion == 'anger':
        sub_dict['anger'] = sub_dict.get('anger') + 1
        dict_emotions[time] = sub_dict 
    elif emotion == 'surprise':
        sub_dict['surprise'] = sub_dict.get('surprise') + 1
        dict_emotions[time] = sub_dict 
    elif emotion == 'sadness':
        sub_dict['sadness'] = sub_dict.get('sadness') + 1
        dict_emotions[time] = sub_dict 
    else:
        sub_dict['fear'] = sub_dict.get('fear') + 1
        dict_emotions[time] = sub_dict 
    
    return dict_emotions

In [7]:
for s in list_emotions_modified:
    emotion = s[2]
    time = s[0]
    #print(time, ' - ', emotion)
    dict_emotions = count_emotions(dict_emotions, emotion, time)

In [13]:
list_x_csv_emotions = []
for e in dict_emotions:
    sub_dict = dict_emotions[e]
    inner_list = [e, sub_dict['joy'], sub_dict['surprise'], sub_dict['sadness'], sub_dict['anger'], sub_dict['fear']]
    list_x_csv_emotions.append(inner_list)

print(len(list_x_csv_emotions))

177


In [14]:
#salvataggio dei dati in csv

header = ['time', 'tot joy', 'tot surprise', 'tot sadness', 'tot anger', 'tot fear']

with open('emotions_x_time.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(list_x_csv_emotions)